Trabalho realizado por: Bárbara Freixo, PG49169

Esta implementação implementa uma classe Dataset e uma classe MLP para classificação binária utilizando a função de ativação sigmoide. A classe MLP define métodos para inicializar a rede neuronal, definir pesos, prever saídas, calcular a função custo, construir o modelo e normalizar dados. A classe Dataset define métodos para ler e escrever conjuntos de dados, obter atributos e rótulos, dividir conjuntos em treino e teste e processar rótulos binários. A função funcao_sigmoid(x) é uma implementação da função de ativação sigmoide.

In [13]:
import numpy as np
from scipy import optimize

class MLP:
    
    # Inicializa a classe MLP
    def __init__(self, conjunto_dados, nos_ocultos = 2, normalizar = False):
        self.Atributos, self.Rotulos = conjunto_dados.getXy()
        # Adiciona uma coluna de 1s para o bias
        self.Atributos = np.hstack((np.ones([self.Atributos.shape[0], 1]), self.Atributos))
        
        self.n = nos_ocultos
        self.P1 = np.zeros([nos_ocultos, self.Atributos.shape[1]])
        self.P2 = np.zeros([1, nos_ocultos + 1])
        
        # Normaliza os dados se normalizar=True
        if normalizar:
            self.normalizar_dados()
        else:
            self.normalizado = False

    # Define os pesos das camadas
    def definir_pesos(self, p1, p2):
        self.P1 = p1
        self.P2 = p2   

    # Realiza a previsão para uma única instância
    def prever(self, instancia):
        x = np.empty([self.Atributos.shape[1]])        
        x[0] = 1
        x[1:] = np.array(instancia[:self.Atributos.shape[1] - 1])
        
        # Normaliza a instância se os dados estiverem normalizados
        if self.normalizado:
            if np.all(self.desvio_padrao != 0): 
                x[1:] = (x[1:] - self.media) / self.desvio_padrao
            else: x[1:] = (x[1:] - self.media)
        
        # Realiza a previsão
        z2 = np.dot(self.P1, x)
        a2 = np.empty([z2.shape[0] + 1])
        a2[0] = 1
        a2[1:] = funcao_sigmoid(z2)
        z3 = np.dot(self.P2, a2)
                        
        return funcao_sigmoid(z3)

    # Calcula a função de custo
    def funcao_custo(self, pesos = None):
        if pesos is not None:
            self.P1 = pesos[:self.n * self.Atributos.shape[1]].reshape([self.n, self.Atributos.shape[1]])
            self.P2 = pesos[self.n * self.Atributos.shape[1]:].reshape([1, self.n + 1])
        
        m = self.Atributos.shape[0]
        z2 = np.dot(self.Atributos, self.P1.T)
        a2 = np.hstack((np.ones([z2.shape[0], 1]), funcao_sigmoid(z2)))
        z3 = np.dot(a2, self.P2.T)
        previsoes = funcao_sigmoid(z3)
        erro_quadratico = (previsoes - self.Rotulos.reshape(m, 1)) ** 2
        resultado = np.sum(erro_quadratico) / (2 * m)
        return resultado

    # Constrói o modelo MLP
    def construir_modelo(self):
        tamanho = self.n * self.Atributos.shape[1] + self.n + 1
        pesos_iniciais = np.random.rand(tamanho)        
        resultado = optimize.minimize(lambda w: self.funcao_custo(w), pesos_iniciais, method='BFGS', 
                                    options={"maxiter":1000, "disp":False} )
        pesos = resultado.x
        self.P1 = pesos[:self.n * self.Atributos.shape[1]].reshape([self.n, self.Atributos.shape[1]])
        self.P2 = pesos[self.n * self.Atributos.shape[1]:].reshape([1, self.n + 1])

    # Normaliza os dados
    def normalizar_dados(self):
          self.media = np.mean(self.Atributos[:, 1:], axis=0)
          self.Atributos[:, 1:] = self.Atributos[:, 1:] - self.media
          self.desvio_padrao = np.std(self.Atributos[:, 1:], axis=0)
          self.Atributos[:, 1:] = self.Atributos[:, 1:] / self.desvio_padrao
          self.normalizado = True

def funcao_sigmoid(x):
  return (1 / (np.exp(-x)+1))

In [14]:
class Dataset:
    
    # constructor
    def __init__(self, filename = None, X = None, Y = None):
        if filename is not None:
            self.readDataset(filename)
        elif X is not None and Y is not None:
            self.X = X
            self.Y = Y
        else:
            self.X = None
            self.Y = None
        
    def readDataset(self, filename, sep = ","):
        data = np.genfromtxt(filename, delimiter=sep)
        self.X = data[:,0:-1]
        self.Y = data[:,-1]
        
    def writeDataset(self, filename, sep = ","):
        fullds = np.hstack( (self.X, self.Y.reshape(len(self.Y),1)))
        np.savetxt(filename, fullds, delimiter = sep)
        
    def getXy (self):
        return self.X, self.Y
    
    def train_test_split(self, p = 0.7):
        from random import shuffle
        ninst = self.X.shape[0]
        inst_indexes = np.array(range(ninst))
        ntr = (int)(p*ninst)
        shuffle(inst_indexes)
        tr_indexes = inst_indexes[1:ntr]
        tst_indexes = inst_indexes[ntr+1:]
        Xtr = self.X[tr_indexes,]
        ytr = self.Y[tr_indexes]
        Xts = self.X[tst_indexes,]
        yts = self.Y[tst_indexes]
        return (Xtr, ytr, Xts, yts) 
    
    def process_binary_y(self):
        y_values = np.unique(self.Y)
        if len(y_values) == 2:
            self.Y = np.where(self.Y == y_values[0], 0, 1)
        else:
            print("Non binary")

# Testes e exemplos para o algoritmo implementado

## Exemplo de uso do código

Esta implementação é um exemplo de como usar as classes implementadas anteriormente. Primeiro, carrega o conjunto de dados Iris e usa apenas as primeiras 100 amostras para criar um objeto Dataset. Em seguida, o conjunto de dados é dividido num conjunto de treino e num conjunto de teste usando o método train_test_split. Depois, é criado um conjunto de dados de treino para o MLP e o modelo MLP é criado e treinado com os dados de treino usando a classe MLP. Por fim, é feita a previsão com o modelo treinado usando os dados de teste e é calculada a acurácia do modelo.

In [15]:
from sklearn.datasets import load_iris

# Carrega o dataset Iris do sklearn
iris = load_iris()
X = iris.data[:100]  # Usamos apenas as primeiras 100 amostras (setosa e versicolor)
y = iris.target[:100]  # Convertido automaticamente para 0 (setosa) e 1 (versicolor)

# Cria um objeto Dataset
dataset = Dataset(X=X, Y=y)

# Divide o conjunto de dados em treino e teste
X_train, y_train, X_test, y_test = dataset.train_test_split(p=0.7)

# Cria um conjunto de dados de treino para o MLP
train_dataset = Dataset(X=X_train, Y=y_train)

# Cria e treina o modelo MLP
mlp = MLP(conjunto_dados=train_dataset, nos_ocultos=5, normalizar=True)
mlp.construir_modelo()

# Faz previsões com o modelo treinado
correct = 0
for i in range(len(X_test)):
    test_instance = X_test[i]
    prediction = mlp.prever(test_instance)
    print(f"Instância de teste: {test_instance}, Saída prevista: {prediction}, Saída real: {y_test[i]}")
    if round(prediction.item()) == y_test[i]:
        correct += 1

accuracy = correct / len(X_test)
print(f"Acurácia: {accuracy * 100:.2f}%")

Instância de teste: [5.5 2.5 4.  1.3], Saída prevista: [0.99640732], Saída real: 1
Instância de teste: [6.6 3.  4.4 1.4], Saída prevista: [0.99641298], Saída real: 1
Instância de teste: [4.8 3.  1.4 0.3], Saída prevista: [0.00615194], Saída real: 0
Instância de teste: [4.9 2.4 3.3 1. ], Saída prevista: [0.98785893], Saída real: 1
Instância de teste: [5.4 3.9 1.7 0.4], Saída prevista: [0.0057354], Saída real: 0
Instância de teste: [6.2 2.2 4.5 1.5], Saída prevista: [0.99660345], Saída real: 1
Instância de teste: [6.7 3.  5.  1.7], Saída prevista: [0.99643431], Saída real: 1
Instância de teste: [5.4 3.  4.5 1.5], Saída prevista: [0.99624766], Saída real: 1
Instância de teste: [5.2 3.4 1.4 0.2], Saída prevista: [0.00583719], Saída real: 0
Instância de teste: [6.1 2.8 4.  1.3], Saída prevista: [0.99636973], Saída real: 1
Instância de teste: [5.5 2.4 3.7 1. ], Saída prevista: [0.99592454], Saída real: 1
Instância de teste: [5.6 3.  4.1 1.3], Saída prevista: [0.99588232], Saída real: 1
Instâ

## Testes "Unittest"

Foi realizado um teste utilizando o unittest para testar a implementação do MLP. Temos então o seguinte teste:

In [16]:
import unittest
import numpy as np

class TestMLP(unittest.TestCase):
    def setUp(self):
        # Inicializa os dados de entrada (X) e saída (y) para o dataset
        self.X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
        self.y = np.array([0, 1, 1, 0])
        # Cria uma instância de Dataset com os dados X e y
        self.dataset = Dataset(X=self.X, Y=self.y)
        # Cria uma instância de MLP com o dataset criado e 2 neurônios na camada oculta
        self.mlp = MLP(self.dataset, nos_ocultos=2)

    def test_prever(self):
        # Define os pesos iniciais da rede MLP
        self.mlp.definir_pesos(
            np.array([[0.5, 0.5, 0.5], [-0.5, -0.5, -0.5]]),
            np.array([[-1, 1, 1]])
        )
        # Realiza previsões com a rede MLP para todas as entradas do dataset
        previsoes = np.array([self.mlp.prever(self.X[i]) for i in range(4)])
        # Arredonda as previsões para inteiros
        previsoes = np.round(previsoes).astype(int)
        # Calcula a acurácia das previsões
        acuracia = np.sum(previsoes == self.y) / len(self.y)
        # Testa se a acurácia é maior ou igual a 0.5
        self.assertGreaterEqual(acuracia, 0.5)

    def test_funcao_custo(self):
        # Define os pesos iniciais da rede MLP
        self.mlp.definir_pesos(
            np.array([[0.5, 0.5, 0.5], [-0.5, -0.5, -0.5]]),
            np.array([[-1, 1, 1]])
        )
        # Calcula o valor da função custo da rede MLP
        custo = self.mlp.funcao_custo()
        # Testa se o valor do custo é menor que 0.5
        self.assertLess(custo, 0.5)

    def test_construir_modelo(self):
        # Treina o modelo da rede MLP
        self.mlp.construir_modelo()
        # Realiza previsões com a rede MLP treinada para todas as entradas do dataset
        previsoes = np.array([self.mlp.prever(self.X[i]) for i in range(4)])
        # Arredonda as previsões para inteiros
        previsoes = np.round(previsoes).astype(int)
        # Calcula a acurácia das previsões
        acuracia = np.sum(previsoes == self.y) / len(self.y)
        # Testa se a acurácia é maior ou igual a 0.5
        self.assertGreaterEqual(acuracia, 0.5)

    def test_normalizar_dados(self):
        # Normaliza os dados de entrada da rede MLP
        self.mlp.normalizar_dados()
        # Testa se a média dos atributos normalizados é próxima de 0
        self.assertTrue(np.allclose(np.mean(self.mlp.Atributos[:, 1:], axis=0), 0))
        # Testa se o desvio padrão dos atributos normalizados é próximo de 1
        self.assertTrue(np.allclose(np.std(self.mlp.Atributos[:, 1:], axis=0), 1))
        
if __name__ == '__main__':
    # Executa todos os testes definidos na classe TestMLP
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

....
----------------------------------------------------------------------
Ran 4 tests in 0.177s

OK
